<a href="https://colab.research.google.com/github/DGkurnia/discuss1/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libary import
import pandas as pd  # Untuk load & mengolah tabel/CSV/Excel/Gsheet
import numpy as np   # Numpy in Array
import matplotlib.pyplot as plt
import seaborn as sns
import shutil   #managing ZIP files
from datetime import datetime # for managing time-related date
#importing tables
dfcustomer = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/customers.csv")
dforder = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/orders.csv")
dfproduct = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/products.csv")
dfsale = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/sales.csv")
#inspecting each df
dfsale.head(10)


,sales_id,order_id,product_id,price_per_unit,quantity,total_price
0,0,1,218,106,2,212.0
1,1,1,481,118,1,118.0
2,2,1,2,96,3,288.0
3,3,1,1002,106,2,212.0
4,4,1,691,113,3,339.0
5,5,1,981,106,3,318.0
6,6,2,915,96,1,96.0
7,7,2,686,113,1,113.0
8,8,2,1091,115,3,345.0
9,9,2,1196,105,1,NaN


1. Preparing dataset

2. Fixing Missing values

In [2]:
#inspecting missing data
dfcustomer.info()
dforder.info()
tnacol = dfcustomer.isna().sum()
print(tnacol)
print("Jumlah duplikasi: ", dfcustomer.duplicated().sum())
print("Jumlah duplikasi: ",dforder.duplicated().sum())
dforder.describe()
dfproduct.info()
print("Jumlah duplikasi: ", dfproduct.duplicated().sum())
dfproduct.describe()
dfsale.isna().sum()
"""
oke, sampai tahap ini, kita telah berhasil mengidentifikasi berbagai masalah pada data yang telah kita kumpulkan. Berikut merupakan rangkumannya.


Tipe data

Missing value (Terdapat 18 missing values pada kolom gender di
customer_df.)

Duplicate data (Terdapat 6 data yang duplikat.)

Inaccurate value Terdapat inaccurate value pada kolom age.

orders_df

Terdapat kesalahan tipe data untuk kolom order_date & delivery_date di product_df

di sales_df

Terdapat 19 missing value pada kolom total_price.
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    1007 non-null   int64 
 1   customer_name  1007 non-null   object
 2   gender         989 non-null    object
 3   age            1007 non-null   int64 
 4   home_address   1007 non-null   object
 5   zip_code       1007 non-null   int64 
 6   city           1007 non-null   object
 7   state          1007 non-null   object
 8   country        1007 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       1000 non-null   int64 
 1   customer_id    1000 non-null   int64 
 2   payment        1000 non-null   int64 
 3   order_date     1000 non-null   object
 4   de

'\noke, sampai tahap ini, kita telah berhasil mengidentifikasi berbagai masalah pada data yang telah kita kumpulkan. Berikut merupakan rangkumannya.\n\n\nTipe data\n\nMissing value (Terdapat 18 missing values pada kolom gender di\ncustomer_df.)\n\nDuplicate data (Terdapat 6 data yang duplikat.)\n\nInaccurate value Terdapat inaccurate value pada kolom age.\n\norders_df\n\nTerdapat kesalahan tipe data untuk kolom order_date & delivery_date di product_df\n\ndi sales_df\n\nTerdapat 19 missing value pada kolom total_price.\n'

3. Handling mistakes in data

In [ ]:
dfcustomer.drop_duplicates(inplace=True)
#print("Jumlah duplikasi: ", dfcustomer.duplicated().sum())
#dfcustomer segment
dfcustomer.gender.value_counts()
#dfcustomer[dfcustomer.gender.isna()]
dfcustomer.fillna(value="Prefer not to say", inplace=True)
#managing incorrect age
dfcustomer.age.replace(dfcustomer.age.max(), 70, inplace=True)
dfcustomer.age.replace(dfcustomer.age.max(), 50, inplace=True)
#dfcustomer.describe()
#order segment
datetime_columns = ["order_date", "delivery_date"]

for column in datetime_columns:
  dforder[column] = pd.to_datetime(dforder[column])
#dforder.info()
#product segment
dfproduct.drop_duplicates(inplace=True)
#print("Jumlah duplikasi: ", dfproduct.duplicated().sum())
#sale segment
dfsale[dfsale.total_price.isna()]
dfsale["total_price"] = dfsale["price_per_unit"] * dfsale["quantity"]
dfsale.isna().sum()

4. Describing code

In [ ]:
#describibng data
#dfcustomer.describe(include="all")
dfcustomer.groupby(by="gender").agg({
    "customer_id": "nunique",
    "age": ["max", "min", "mean", "std"]
})
#segmenting
dfcustomer.groupby(by="city").customer_id.nunique().sort_values(ascending=False)
dfcustomer.groupby(by="state").customer_id.nunique().sort_values(ascending=False)

describing data - part 2

In [ ]:
#intro
delivery_time = dforder["delivery_date"] - dforder["order_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
dforder["delivery_time"] = round(delivery_time/86400)
#inspecting
#print(dforder["delivery_time"])
#customer inspection
customer_id_in_orders_df =  dforder.customer_id.tolist()
dfcustomer["status"] = dfcustomer["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")
dfcustomer.groupby(by="status").customer_id.count()
#merging dataset
orders_customers_df = pd.merge(
    left= dforder,
    right= dfcustomer,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
orders_customers_df.head()

Pivot segmentation

In [ ]:
#city-based pivot
orders_customers_df.groupby(by="city").order_id.nunique().sort_values(ascending=False).reset_index().head(10)
#state based pivot
orders_customers_df.groupby(by="state").order_id.nunique().sort_values(ascending=False)
#gender based
orders_customers_df.groupby(by="gender").order_id.nunique().sort_values(ascending=False)
#age based group
orders_customers_df["age_group"] = orders_customers_df.age.apply(lambda x: "Youth" if x <= 24 else ("Seniors" if x > 64 else "Adults"))
orders_customers_df.groupby(by="age_group").order_id.nunique().sort_values(ascending=False)

product and sales

In [ ]:
#sales prediction (phase 1)
dfproduct.sort_values(by="price", ascending=False)
#pivot in sales and product
dfproduct.groupby(by="product_type").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price":  ["min", "max"]
})

#determining
dfproduct.groupby(by="product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})
#merging the dataset
sales_product_df = pd.merge(
    left= dfsale,
    right= dfproduct,
    how="left",
    left_on="product_id",
    right_on="product_id"
)
sales_product_df.head(10)

Pivot table from sales and products

In [ ]:
#preparing pivot from product typer
sales_product_df.groupby(by="product_type").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
})
#preparing pivot from name
sales_product_df.groupby(by="product_name").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
}).sort_values(by="total_price", ascending=False)

Merging all dataset

In [ ]:
#first step
all_df = pd.merge(
    left = sales_product_df,
    right = orders_customers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
#all_df.head(10)
all_df.groupby(by=["state", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})
all_df.groupby(by=["gender", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

all_df.groupby(by=["age_group", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

Visualisation

In [ ]:
#initial
monthly_orders_df = all_df.resample(rule='M', on='order_date').agg({
    "order_id": "nunique",
    "total_price": "sum"
})
monthly_orders_df.index = monthly_orders_df.index.strftime('%Y-%m')
monthly_orders_df = monthly_orders_df.reset_index()
monthly_orders_df.rename(columns={
    "order_id": "order_count",
    "total_price": "revenue"
}, inplace=True)
#monthly_orders_df.head()
#displaying chart
monthly_orders_df = all_df.resample(rule='M', on='order_date').agg({
    "order_id": "nunique",
    "total_price": "sum"
})
monthly_orders_df.index = monthly_orders_df.index.strftime('%B') #mengubah format order date menjadi nama bulan

monthly_orders_df = monthly_orders_df.reset_index()
monthly_orders_df.rename(columns={
    "order_id": "order_count",
    "total_price": "revenue"
}, inplace=True)

plt.figure(figsize=(10, 5))
plt.plot(monthly_orders_df["order_date"], monthly_orders_df["order_count"], marker='o', linewidth=2, color="#72BCD4")
plt.title("Number of Orders per Month (2021)", loc="center", fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

Detailed chart for revenue

In [ ]:
#displaying size
plt.figure(figsize=(12, 6))
plt.plot(
    monthly_orders_df["order_date"],
    monthly_orders_df["revenue"],
    marker='o',
    linewidth=2,
    color="#72BCD4"
)
plt.title("Total Revenue per Month (2021)", loc="center", fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

determining the most and the least amount of program

In [13]:
#the most and least amount product
sum_order_items_df = all_df.groupby("product_name").quantity_x.sum().sort_values(ascending=False).reset_index()
sum_order_items_df.head(15)

,product_name,quantity_x
0,Denim,527
1,Joggers,334
2,Pleated,308
3,Casual Slim Fit,306
4,Shearling,302
5,Relaxed Leg,301
6,Trench Coat,299
7,Puffer,298
8,Bomber,297
9,Chambray,290
